# Bibliotecas utilizadas

In [1]:
# Bibliotecas utilizadas
!pip install py7zr
!pip install minio
from minio import Minio
from minio.error import S3Error

import numpy as np
import pandas as pd
from ftplib import FTP
import os, sys, os.path
import py7zr
import glob

     |████████████████████████████████| 65 kB 1.6 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 4.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 14.9 MB/s eta 0:00:01
     |████████████████████████████████| 122 kB 46.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 38.4 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 1.6 MB/s eta 0:00:01


In [18]:
from importlib.metadata import version
import minio
print(minio.__version__)

7.1.0


# Tratamento dos Dados - Pré-processamento e limpeza

#### Realizar o download dos arquivos via FTP

* Conectar ao servidor FTP.
* Listar as pastas do diretório remoto.
* Criar as pastas mantendo a estrutura da origem (anos).
* Informar na chamada da função o ano de inicio e fim que devem ser processados.
* Realiza o download dos arquivos na pasta referente ao ano.

#### Bibliotecas utilizadas:

* ftplib
* os, sys, os.path


In [2]:
# Download dos arquivos FTP:
# Informar o servidor ftp, diretorio (source) dos arquivos/diretorios, 
# diretorio local onde os arquivos serao baixados
# ano de inicio e fim

def downloadRais(ftp_server, ftp_directory, local_directory, year_start, year_end):
    
    ftp_server = ftp_server
    ftp_directory = ftp_directory
    local_directory = local_directory
    year_start = year_start
    year_end = year_end
    
    # Criar diretorio se não existir:
    try:
        os.stat(local_directory)
        #print("Directory found", local_directory)
    except:
        #print("Creating Directory: ", local_directory)
        os.mkdir(local_directory)



    ftp = FTP(ftp_server)
    ftp.login()

    for year in range(year_start, year_end+1):    
        print("Ano:  ",str(year))
        # Criar diretorio local para cada ano onde os arquivos serao armazenados
        local_path = local_directory + str(year) + "/"
        #print(local_path)
        try:
            os.stat(local_path)
            #print("Directory found", local_path)
        except:
            #print("Creating Directory: ", local_path)
            os.mkdir(local_path)
        
        # Listar arquivos no diretorio remoto (FTP)
        ftp_path = ftp_directory + str(year) + "/"
        #print(ftp_path)
        ftp.cwd(ftp_path)
        ftp_files = ftp.nlst()
        #print(ftp_files)
    
        # Download dos arquivos (FTP)
        for file in ftp_files:
            remote_filename = ftp_path + file
            print("Downloading file: ",remote_filename)
            local_filename = local_path + file
            #print("Save as: ",local_filename)
        
            file = open(local_filename, 'wb')
            ftp.retrbinary("RETR " + remote_filename, file.write)
            #print("Closing file")
            file.close()
    print("Download FTP Completed.")
    return()
        




#### Extrair arquivos 7zip

* Listar os arquivos .7z
* Extair os arquivos (.txt)
* Remove o arquivo .7z


#### Bibliotecas utilizadas:

* py7zr
* glob
* os, sys, os.path


In [3]:
# Descompactar arquivos 7z e remover arquivo
# Informar o caminho completo dos diretorios

def extractRais(local_dir):
    local_dir = local_dir
    os.chdir(local_dir)
    
    dirlist = os.listdir()
    #print(dirlist)
    for d in dirlist:
        os.chdir(local_dir+d)
        print("Extracting 7z files from directory: ", os.getcwd())
        for file in glob.glob("*.7z"):
            #print("Extracting file: ",file)
            with py7zr.SevenZipFile(file, 'r') as zipfile:
                zipfile.extract()
            #print("Removing file 7z: ", file)
            os.remove(file)
    print("Extraction Completed")
    return()

    

#### Envio de Arquivos para o MinIO - Datalake

* Após pré-processamento dos dados, envia os arquivos enriquecidos no formato parquet para o bucket no MinIO.


#### Bibliotecas utilizadas:

* minio

In [4]:
# Função para enviar arquivos .parquet para o MinIO
def uploadMinIO(ano, source_file, target_filename):
    
    ano = ano
    source_file = source_file
    target_filename = str(ano)+"/"+target_filename
    bucket = "a3datanew"
    
    # Create a client with the MinIO server playground, its access key
    # and secret key.
    client = Minio(
        "172.17.0.3:9000",
        access_key="",
        secret_key="",
        secure=False
    )

    # Criar bucket caso não exista
    found = client.bucket_exists(bucket)
    if not found:
        client.make_bucket(bucket)

    # Enviando arquivo para o MinIO
    client.fput_object(bucket, target_filename, source_file )
    print("Arquivo enviado para o MinIO:" , target_filename)
    
    


#### Processamento e enriquecimento dos dados

* Arquivos txt contém dados separados por ;
* Identificamos problemas na codificação dos dados (acentuação). Para corrigir o problema, utilizamos encoding='latin-1'.
* Para responder as questões foram selecionadas as seguintes colunas:
    - 'CBO Ocupação 2002', 'Qtd Hora Contr', 'Município', 'Vl Remun Média Nom', 'Sexo Trabalhador' ,'Tipo Vínculo', 'CNAE 2.0 Classe', 'CNAE 95 Classe', 'Escolaridade após 2005'
    
    
* O formato do dado 'Vl Remun Média Nom' precisou ser corrigido. Alteramos o separador decimal (,) para (.)


#### Novas colunas criadas/modificadas:

* ANO : Utilizamos o nome da pasta para obter o valor do ano.
* 'Sexo Trabalhador' : Alteramos os valores numéricos (1, 2, -1) para character (M, F, I)
* 'CBO Ocupação 2002' : Dados referentes a ocupação, de acordo com o Cadastro Brasileiro de Ocupações. Alguns códigos apresentavam caracter "-", que foram removidos durante o pré-processamento.
* Nova coluna com a sigla dos estados. Para criação desta coluna foi realizado um join com os dados extraídos da sheet "municipio" (arquivo layout encontrado no FTP).
* Nova coluna Regioes com base nos estados.
* Nova  coluna ocupacao com base nos códigos de ocupação encontrados na coluna "CBO Ocupação 2002" e sheet "ocupacao" da planilha layout.
* Nova coluna vinculo com base na coluna "Tipo Vínculo"
* Nova coluna com grau_instrucao de acordo com a coluna "Escolaridade após 2005"
* Nova coluna INDUSTRIA contendo os setores/atividades economicas de acordo com o cógico do CNAE encontrado na coluna "CNAE 2.0 Classe"

** Para realizar algumas das alterações e criação de novas colunas foram utilizados dados extraídos da planilha Layout (encontradas no FTP), e para a definição da coluna dados estraídos do site do IBGE - Concla (https://concla.ibge.gov.br/busca-online-cnae.html?view=estrutura)




In [5]:
def dataMunging(file_path, ano, file):
    
    file_path = file_path
    ano = ano
    file = file
    
    col_list = ['CBO Ocupação 2002', 'Qtd Hora Contr', 'Município', 'Vl Remun Média Nom', 'Sexo Trabalhador'
           ,'Tipo Vínculo', 'CNAE 2.0 Classe', 'CNAE 95 Classe', 'Escolaridade após 2005', 'Vínculo Ativo 31/12']
    
    df = pd.read_csv(file_path, usecols=col_list, sep=";"
                      ,encoding='latin-1', low_memory=False) #, nrows=10000) # tirar nrows
    
    # Corrigir formato dado coluna 'Vl Remun Média Nom'
    df['Vl Remun Média Nom'] = df['Vl Remun Média Nom'].apply(lambda x: x.replace(',', '.')).astype(float)
    
    # Criar coluna com o ano do arquivo
    df["ANO"] = ano
    
    # Alterar Sexo para M, F, I
    conditions = [(df['Sexo Trabalhador'] == 1),
              (df['Sexo Trabalhador'] == 2),
              (df['Sexo Trabalhador'] == -1)]
    values = ['M', 'F', 'I']
    df['Sexo Trabalhador'] = np.select(conditions, values)
    
    # Corrigir formato dados coluna ocupacao
    df['CBO Ocupação 2002'] = df['CBO Ocupação 2002'].apply(lambda x: str(x).replace('-','')).astype(int)

    
    # Novas colunas estado e regiao
    municipios = pd.read_csv('/home/jovyan/work/municipios.csv', sep=";")
    
    # Adicionar coluna estado (join municipios)
    df_municipios = municipios[['cod_mun','estado']] # selecionar apenas as colunas cod_mun e estado
    df = df.join(df_municipios.set_index('cod_mun'), how="left", on="Município" )
    
    # Alterar estados para Uppercase
    df['estado'] = df['estado'].apply(lambda x: x.upper())
    
    # Criar coluna Regioes com base nos estados:
    conditions = [(df['estado'] == 'PR') | (df['estado'] == 'SC') | (df['estado'] == 'RS'),
                  (df['estado'] == 'SP') | (df['estado'] == 'MG') | (df['estado'] == 'RJ') | (df['estado'] == 'DF'),
                  (df['estado'] == 'BA') | (df['estado'] == 'PI') | (df['estado'] == 'MA') | (df['estado'] == 'CE') | 
                   (df['estado'] == 'RN') | (df['estado'] == 'PB') | (df['estado'] == 'AL') | (df['estado'] == 'SE'),
                  (df['estado'] == 'MT') | (df['estado'] == 'GO') | (df['estado'] == 'MS') | (df['estado'] == 'DF'),
                  (df['estado'] == 'RO') | (df['estado'] == 'AC') | (df['estado'] == 'AM') | (df['estado'] == 'PA') |
                   (df['estado'] == 'AP') | (df['estado'] == 'RR') | (df['estado'] == 'TO')] 
    values = ['SUL', 'SUDESTE', 'NORDESTE', 'CENTRO-OESTE', 'NORTE']
    df["REGIAO"] = np.select(conditions, values)
    
    # Criar nova coluna com o nome dos estados (para melhorar as visualizações)
    df_nomes_estados = pd.read_csv('/home/jovyan/work/nomes_estados.csv', sep=";", usecols=['Sigla', 'Estado'])
    df = df.join(df_nomes_estados.set_index('Sigla'), on="estado", how="left")
    df.rename(columns = {'Estado': 'ESTADO_NOME'}, inplace = True)
    
    
    # Nova coluna ocupacao
    df_ocupacao = pd.read_csv('/home/jovyan/work/ocupacao.csv', sep=";")
    df = df.join(df_ocupacao.set_index('cod_ocup'), how="left", on="CBO Ocupação 2002")
    
    # Nova coluna vinculo
    df_vinculos = pd.read_csv('/home/jovyan/work/vinculos.csv', sep=";")
    df = df.join(df_vinculos.set_index('cod_vinculo'), how="left", on="Tipo Vínculo")
    
    # Nova coluna grau instrucao
    df_escolaridade = pd.read_csv('/home/jovyan/work/grau_instrucao.csv', sep=";")
    df = df.join(df_escolaridade.set_index('cod_instrucao'), how="left", on="Escolaridade após 2005")
    
    conditions = [(df['CNAE 2.0 Classe'] >= 1000) &  (df['CNAE 2.0 Classe'] <= 3999),
                  (df['CNAE 2.0 Classe'] >= 5000) &  (df['CNAE 2.0 Classe'] <= 9999),
                  (df['CNAE 2.0 Classe'] >= 10000) &  (df['CNAE 2.0 Classe'] <= 33999),
                  (df['CNAE 2.0 Classe'] >= 35000) &  (df['CNAE 2.0 Classe'] <= 35999),
                  (df['CNAE 2.0 Classe'] >= 36000) &  (df['CNAE 2.0 Classe'] <= 39999),
                  (df['CNAE 2.0 Classe'] >= 41000) &  (df['CNAE 2.0 Classe'] <= 43999),
                  (df['CNAE 2.0 Classe'] >= 45000) &  (df['CNAE 2.0 Classe'] <= 47999),
                  (df['CNAE 2.0 Classe'] >= 49000) &  (df['CNAE 2.0 Classe'] <= 53999),
                  (df['CNAE 2.0 Classe'] >= 55000) &  (df['CNAE 2.0 Classe'] <= 56999),
                  (df['CNAE 2.0 Classe'] >= 58000) &  (df['CNAE 2.0 Classe'] <= 63999),
                  (df['CNAE 2.0 Classe'] >= 64000) &  (df['CNAE 2.0 Classe'] <= 66999),
                  (df['CNAE 2.0 Classe'] >= 68000) &  (df['CNAE 2.0 Classe'] <= 68999),
                  (df['CNAE 2.0 Classe'] >= 69000) &  (df['CNAE 2.0 Classe'] <= 75999),
                  (df['CNAE 2.0 Classe'] >= 77000) &  (df['CNAE 2.0 Classe'] <= 82999),
                  (df['CNAE 2.0 Classe'] >= 84000) &  (df['CNAE 2.0 Classe'] <= 84999),
                  (df['CNAE 2.0 Classe'] >= 85000) &  (df['CNAE 2.0 Classe'] <= 85999),
                  (df['CNAE 2.0 Classe'] >= 86000) &  (df['CNAE 2.0 Classe'] <= 88999),
                  (df['CNAE 2.0 Classe'] >= 90000) &  (df['CNAE 2.0 Classe'] <= 93999),
                  (df['CNAE 2.0 Classe'] >= 94000) &  (df['CNAE 2.0 Classe'] <= 96999),
                  (df['CNAE 2.0 Classe'] >= 97000) &  (df['CNAE 2.0 Classe'] <= 97999),
                  (df['CNAE 2.0 Classe'] >= 99000) &  (df['CNAE 2.0 Classe'] <= 99999)]
    
    values = ['AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PESCA E AQÜICULTURA', 
              'INDÚSTRIAS EXTRATIVAS', 'INDÚSTRIAS DE TRANSFORMAÇÃO', 'ELETRICIDADE E GÁS', 
              'ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS E DESCONTAMINAÇÃO', 'CONSTRUÇÃO', 
              'COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E MOTOCICLETAS', 
              'TRANSPORTE, ARMAZENAGEM E CORREIO', 'ALOJAMENTO E ALIMENTAÇÃO', 
              'INFORMAÇÃO E COMUNICAÇÃO', 'ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS', 
              'ATIVIDADES IMOBILIÁRIAS', 'ATIVIDADES PROFISSIONAIS, CIENTÍFICAS E TÉCNICAS', 
              'ATIVIDADES ADMINISTRATIVAS E SERVIÇOS COMPLEMENTARES', 
              'ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL', 'EDUCAÇÃO', 
              'SAÚDE HUMANA E SERVIÇOS SOCIAIS', 'ARTES, CULTURA, ESPORTE E RECREAÇÃO', 
              'OUTRAS ATIVIDADES DE SERVIÇOS', 'SERVIÇOS DOMÉSTICOS', 
              'ORGANISMOS INTERNACIONAIS E OUTRAS INSTITUIÇÕES EXTRATERRITORIAIS']
    
    df["INDUSTRIA"] = np.select(conditions, values)

    
    # Salvar em formato parquet
    # Criar diretorio enriched se não existir:
    enriched = '/home/jovyan/work/enriched/'
    try:
        os.stat(enriched)
        #print("Directory found", enriched)
    except:
        #print("Creating Directory: ", enriched)
        os.mkdir(enriched)

    # Criar diretorio para o ano dentro da pasta enriched
    enriched_path = enriched+ano+'/'
    try:
        os.stat(enriched_path)
        #print("Directory found", enriched_path)
    except:
        #print("Creating Directory: ", enriched_path)
        os.mkdir(enriched_path)
    
    # Salvar arquivo parquet
    os.chdir(enriched_path)
    
    # Pegar nome do arquivo sem o .txt e adicionar extensao .parquet
    nome = file.split('.')
    nome_arquivo = enriched_path + nome[0]+".parquet"
    df.to_parquet(nome_arquivo)
    
    # Enviar arquivos .parquet processados para o MinIO
    file_minio = nome[0]+".parquet"
    uploadMinIO(ano, nome_arquivo, file_minio)
    
    return()

In [6]:
def startDataProc(ftp_server, ftp_directory, local_directory, year_start, year_end):
    print("Start - FTP Download")
    #downloadRais(ftp_server, ftp_directory, local_directory, year_start, year_end)
    
    print("Start - Extract of 7z files")
    #extractRais(local_directory)
    
    # Listar arquivos e pastas e iniciar o pre-processamento
    print("Start - Data Processing")
    os.chdir(local_directory)
    anos = os.listdir()
    #for a in anos:
    for a in range(year_start, year_end+1):
        #os.chdir(local_directory+a)
        os.chdir(local_directory+str(a))
        full_path = os.getcwd() + "/"
        for file in glob.glob("*.txt"):
            if not("ESTB" and "ESTAB") in file:
                print(file)
                file_path = full_path + file
                a = str(a)
                dataMunging(file_path, a, file)

    return()

In [9]:
ftp_server = "ftp.mtps.gov.br"
ftp_directory = "/pdet/microdados/RAIS/"
local_directory = "/home/jovyan/work/data2/"
year_start = 2010
year_end = 2019

startDataProc(ftp_server, ftp_directory, local_directory, year_start, year_end)

Start - FTP Download
Start - Extract of 7z files
Start - Data Processing
RAIS_VINC_PUB_CENTRO_OESTE.txt
Arquivo enviado para o MinIO: 2019/RAIS_VINC_PUB_CENTRO_OESTE.parquet
RAIS_VINC_PUB_MG_ES_RJ.txt
Arquivo enviado para o MinIO: 2019/RAIS_VINC_PUB_MG_ES_RJ.parquet
RAIS_VINC_PUB_NORDESTE.txt
Arquivo enviado para o MinIO: 2019/RAIS_VINC_PUB_NORDESTE.parquet
RAIS_VINC_PUB_NORTE.txt
Arquivo enviado para o MinIO: 2019/RAIS_VINC_PUB_NORTE.parquet
RAIS_VINC_PUB_SP.txt
Arquivo enviado para o MinIO: 2019/RAIS_VINC_PUB_SP.parquet
RAIS_VINC_PUB_SUL.txt
Arquivo enviado para o MinIO: 2019/RAIS_VINC_PUB_SUL.parquet


()

# Exploração dos dados enriquecidos

In [23]:
df2 = pd.read_csv('/home/jovyan/work/data2/2019/RAIS_VINC_PUB_NORTE.txt', sep=";", encoding='latin-1', low_memory=False)
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3669279 entries, 0 to 3669278
Data columns (total 60 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   Bairros SP               object
 1   Bairros Fortaleza        object
 2   Bairros RJ               object
 3   Causa Afastamento 1      int64 
 4   Causa Afastamento 2      int64 
 5   Causa Afastamento 3      int64 
 6   Motivo Desligamento      int64 
 7   CBO Ocupação 2002        object
 8   CNAE 2.0 Classe          int64 
 9   CNAE 95 Classe           int64 
 10  Distritos SP             object
 11  Vínculo Ativo 31/12      int64 
 12  Faixa Etária             int64 
 13  Faixa Hora Contrat       int64 
 14  Faixa Remun Dezem (SM)   int64 
 15  Faixa Remun Média (SM)   int64 
 16  Faixa Tempo Emprego      int64 
 17  Escolaridade após 2005   int64 
 18  Qtd Hora Contr           int64 
 19  Idade                    int64 
 20  Ind CEI Vinculado        int64 
 21  Ind Simples              int64 

In [19]:
df2["Vínculo Ativo 31/12"].unique()

array([0, 1])

In [16]:
df2.head()

,Bairros SP,Bairros Fortaleza,Bairros RJ,Causa Afastamento 1,Causa Afastamento 2,Causa Afastamento 3,Motivo Desligamento,CBO Ocupação 2002,CNAE 2.0 Classe,CNAE 95 Classe,...,Vl Rem Maio CC,Vl Rem Junho CC,Vl Rem Julho CC,Vl Rem Agosto CC,Vl Rem Setembro CC,Vl Rem Outubro CC,Vl Rem Novembro CC,Ano Chegada Brasil,Ind Trab Intermitente,Ind Trab Parcial
0,{ñ class},{ñ class},{ñ class},99,99,99,11,717020,81125,70408,...,"000000013,70","000000013,70","000000013,70","000000000,00","000000000,00","000000000,00","000000000,00",0,0,0
1,{ñ class},{ñ class},{ñ class},99,99,99,11,514320,33147,29920,...,"000000011,28","000000011,28","000000011,28","000000011,28","000000011,28","000000011,28","000000000,00",0,0,0
2,{ñ class},{ñ class},{ñ class},99,99,99,21,715115,47440,52442,...,"000000000,00","000000000,00","000000000,00","000000000,00","000000000,00","000000000,00","000000016,19",0,0,0
3,{ñ class},{ñ class},{ñ class},99,99,99,11,724510,46796,51535,...,"000000000,00","000000000,00","000000000,00","000000000,00","000000000,00","000000000,00","000000000,00",0,0,0
4,{ñ class},{ñ class},{ñ class},99,99,99,11,862120,16102,20109,...,"000001596,80","000001596,80","000001596,80","000000770,87","000000000,00","000000000,00","000000000,00",0,0,0


In [24]:
df2 = pd.read_parquet('/home/jovyan/work/enriched/2019/RAIS_VINC_PUB_NORTE.parquet')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3669279 entries, 0 to 3669278
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   CBO Ocupação 2002       int64  
 1   CNAE 2.0 Classe         int64  
 2   CNAE 95 Classe          int64  
 3   Escolaridade após 2005  int64  
 4   Qtd Hora Contr          int64  
 5   Município               int64  
 6   Vl Remun Média Nom      float64
 7   Sexo Trabalhador        object 
 8   Tipo Vínculo            int64  
 9   ANO                     object 
 10  estado                  object 
 11  REGIAO                  object 
 12  ESTADO_NOME             object 
 13  ocupacao                object 
 14  vinculo                 object 
 15  grau_instrucao          object 
 16  INDUSTRIA               object 
dtypes: float64(1), int64(7), object(9)
memory usage: 475.9+ MB


In [ ]:
# Trabalho intermitente: http://www.rais.gov.br/sitio/rais_ftp/ManualRAIS2020.pdf
# Horas Contrato = 1

df2[df2['Qtd Hora Contr'] == 1]

In [ ]:
# Tecnologia (https://concla.ibge.gov.br/busca-online-cnae.html?view=grupo&tipo=cnae&versao=7&grupo=620)
df2[(df2['CNAE 2.0 Classe'] >= 62000) & (df2['CNAE 2.0 Classe'] <= 62099) ]


In [ ]:
# SAUDE (https://concla.ibge.gov.br/busca-online-cnae.html?view=divisao&tipo=cnae&versao=7&divisao=86)
df2[(df2['CNAE 2.0 Classe'] >= 86100) & (df2['CNAE 2.0 Classe'] <= 86999) ]


In [ ]:
# AUTOMOBILISTICO (https://concla.ibge.gov.br/busca-online-cnae.html?view=divisao&tipo=cnae&versao=7&divisao=29)
df2[(df2['CNAE 2.0 Classe'] >= 29100) & (df2['CNAE 2.0 Classe'] <= 29599) ]

In [ ]:
# AGRONEGOCIO (https://concla.ibge.gov.br/busca-online-cnae.html?view=secao&tipo=cnae&versaoclasse=7&secao=A)
df2[(df2['CNAE 2.0 Classe'] >= 1111 ) & (df2['CNAE 2.0 Classe'] <= 3221) ]